In [1]:
!pip install mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.7 MB/s eta 0:00:00
  Created wheel for mljar-supervised: filename=mljar_supervised-0.11.5-py3-none-any.whl size=147060 sha256=6f9a6b313a7e2d8323f354d4b6a8130626d523ea972504962abb5ff026ecfa64
  Stored in directory: /root/.cache/pip/whe

In [12]:
import pandas as pd
import numpy as np
import random
import os
from supervised.automl import AutoML
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import random
import os
import gc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # 행운의 seed

In [14]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [15]:
# ID 열 제거
train_df = train_df.drop('ID', axis=1)
test_df = test_df.drop('ID', axis=1)

In [16]:
# Weight_Status, Gender 열을 숫자 데이터로 변환
train_df['Weight_Status'] = train_df['Weight_Status'].map({'Normal Weight': 0, 'Overweight': 1, 'Obese': 2})
train_df['Gender'] = train_df['Gender'].map({'M': 0, 'F': 1})
test_df['Weight_Status'] = test_df['Weight_Status'].map({'Normal Weight': 0, 'Overweight': 1, 'Obese': 2})
test_df['Gender'] = test_df['Gender'].map({'M': 0, 'F': 1})

In [17]:
# PolynomialFeatures를 사용하여 데이터 전처리
poly = PolynomialFeatures(degree=2, include_bias=False)
X = poly.fit_transform(train_df.drop('Calories_Burned', axis=1))
y = train_df['Calories_Burned']

In [20]:
test_df_poly = poly.fit_transform(test_df)

In [22]:
test_df_poly

array([[2.600e+01, 1.051e+02, 1.070e+02, ..., 1.000e+00, 4.500e+01,
        2.025e+03],
       [2.900e+01, 1.049e+02, 1.110e+02, ..., 0.000e+00, 0.000e+00,
        4.410e+02],
       [1.100e+01, 1.040e+02, 9.000e+01, ..., 0.000e+00, 0.000e+00,
        3.364e+03],
       ...,
       [2.400e+01, 1.051e+02, 9.700e+01, ..., 1.000e+00, 4.200e+01,
        1.764e+03],
       [8.000e+00, 1.031e+02, 8.300e+01, ..., 1.000e+00, 4.800e+01,
        2.304e+03],
       [2.900e+01, 1.056e+02, 1.070e+02, ..., 1.000e+00, 4.400e+01,
        1.936e+03]])

In [18]:
# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
X_test = scaler.fit_transform(test_df_poly)

In [24]:
X_test

array([[ 1.26043576,  0.7484879 ,  1.19644336, ...,  0.99097407,
         0.94565196, -0.06836125],
       [ 1.62262994,  0.60479661,  1.61401124, ..., -1.00910814,
        -0.87777196, -1.04634641],
       [-0.55053516, -0.04181416, -0.57822018, ..., -1.00910814,
        -0.87777196,  0.75835727],
       ...,
       [ 1.01897297,  0.7484879 ,  0.15252363, ...,  0.99097407,
         0.82409036, -0.22950653],
       [-0.91272934, -0.68842494, -1.30896398, ...,  0.99097407,
         1.06721355,  0.10389751],
       [ 1.62262994,  1.10771611,  1.19644336, ...,  0.99097407,
         0.90513142, -0.12331117]])

In [29]:
automl = AutoML(mode="Compete", eval_metric='rmse',total_time_limit=3600)

In [ ]:
automl.fit(X,y)

AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 16.137708 trained in 1.68 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree rmse 15.516733 trained in 9.97 seconds
2_DecisionTree rmse 12.365723 trained in 9.32 seconds
3_DecisionTree rmse 12.365723 trained in 9.1 seconds
4_Line

In [ ]:
preds = automl.predict(test_x)

In [ ]:
preds

array([0.57471465, 0.63229731, 0.17586672, ..., 0.43171307, 0.10469064,
       0.63531938])

In [ ]:
y_train_orig = tmp_scaler.inverse_transform(preds.reshape(-1, 1))

In [ ]:
y_train_orig

array([[172.83967945],
       [190.05689538],
       [ 53.58414909],
       ...,
       [130.0822091 ],
       [ 32.30250154],
       [190.96049395]])

In [ ]:
original_predictions = np.expm1(preds)



In [ ]:
original_predictions 

array([173.30731769, 189.58189392,  52.47635869, ..., 130.3391145 ,
        31.83627088, 191.44091451])

In [ ]:
original_predictions

array([172.91414245, 189.659026  ,  53.38801413, ..., 130.20022219,
        32.18654902, 191.26626596])

In [ ]:
original_y_pred = minmax_scaler.inverse_transform(preds.reshape(-1, 1))

In [ ]:
original_y_pred 

In [ ]:
original_predictions = np.expm1(original_y_pred)

In [ ]:
original_predictions 

In [ ]:
original_predictions

In [ ]:
import gc
gc.collect()

In [ ]:
submission = pd.read_csv('/content/sample_submission.csv')

In [ ]:
submission['Calories_Burned'] = y_train_orig

In [ ]:
submission.to_csv('/content/scale.csv', index = False)